In [1]:
param_date = '20240706'

In [2]:
import os

list_file = sorted(list(os.walk(f'./{param_date}/12.vehicle_1'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['part_1-fragment_1.xlsx',
 'part_10-fragment_10.xlsx',
 'part_11-fragment_11.xlsx',
 'part_12-fragment_12.xlsx',
 'part_13-fragment_13.xlsx',
 'part_14-fragment_14.xlsx',
 'part_15-fragment_15.xlsx',
 'part_16-fragment_16.xlsx',
 'part_17-fragment_17.xlsx',
 'part_18-fragment_18.xlsx',
 'part_19-fragment_19.xlsx',
 'part_2-fragment_2.xlsx',
 'part_20-fragment_20.xlsx',
 'part_21-fragment_21.xlsx',
 'part_22-fragment_22.xlsx',
 'part_23-fragment_23.xlsx',
 'part_24-fragment_24.xlsx',
 'part_25-fragment_25.xlsx',
 'part_26-fragment_26.xlsx',
 'part_27-fragment_27.xlsx',
 'part_28-fragment_28.xlsx',
 'part_29-fragment_29.xlsx',
 'part_3-fragment_3.xlsx',
 'part_30-fragment_30.xlsx',
 'part_31-fragment_31.xlsx',
 'part_32-fragment_32.xlsx',
 'part_33-fragment_33.xlsx',
 'part_4-fragment_4.xlsx',
 'part_5-fragment_5.xlsx',
 'part_6-fragment_6.xlsx',
 'part_7-fragment_7.xlsx',
 'part_8-fragment_8.xlsx',
 'part_9-fragment_9.xlsx']

In [3]:
from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./{param_date}/12.vehicle_1/{file}',
              f'''./{param_date}/12.vehicle_1/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

Progress: 100%|█████████████████████████████| 33/33 [00:00<00:00, 905.43it/s]

Done ~


In [4]:
import os

list_file = sorted(list(os.walk(f'./{param_date}/12.vehicle_1'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['part_1-fragment_1.xlsx',
 'part_10-fragment_10.xlsx',
 'part_11-fragment_11.xlsx',
 'part_12-fragment_12.xlsx',
 'part_13-fragment_13.xlsx',
 'part_14-fragment_14.xlsx',
 'part_15-fragment_15.xlsx',
 'part_16-fragment_16.xlsx',
 'part_17-fragment_17.xlsx',
 'part_18-fragment_18.xlsx',
 'part_19-fragment_19.xlsx',
 'part_2-fragment_2.xlsx',
 'part_20-fragment_20.xlsx',
 'part_21-fragment_21.xlsx',
 'part_22-fragment_22.xlsx',
 'part_23-fragment_23.xlsx',
 'part_24-fragment_24.xlsx',
 'part_25-fragment_25.xlsx',
 'part_26-fragment_26.xlsx',
 'part_27-fragment_27.xlsx',
 'part_28-fragment_28.xlsx',
 'part_29-fragment_29.xlsx',
 'part_3-fragment_3.xlsx',
 'part_30-fragment_30.xlsx',
 'part_31-fragment_31.xlsx',
 'part_32-fragment_32.xlsx',
 'part_33-fragment_33.xlsx',
 'part_4-fragment_4.xlsx',
 'part_5-fragment_5.xlsx',
 'part_6-fragment_6.xlsx',
 'part_7-fragment_7.xlsx',
 'part_8-fragment_8.xlsx',
 'part_9-fragment_9.xlsx']

In [5]:
import pandas as pd

output = pd.DataFrame()
for file in list_file:
    print(file)
    input_ = pd.read_excel(f'./{param_date}/12.vehicle_1/{file}',
                           header=0,
                           dtype=str).fillna('')

    # = = = = = = = = = = = = = = =

    input_2 = input_[['Part Number']].drop_duplicates(keep='first', ignore_index=True)
    input_2 = input_2.sort_values(by=['Part Number'],
                                  ascending=[True],
                                  ignore_index=True)

    # = = = = = = = = = = = = = = =

    for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
        df_temp = input_[input_['Part Number'] == input_2.loc[i, 'Part Number']].copy().reset_index(drop=True)

        # = = = = = = = = = = = = = = =

        dict_vehicle = {}
        for j in range(len(df_temp)):
            if df_temp.loc[j, 'Make'] not in dict_vehicle.keys():
                dict_vehicle[df_temp.loc[j, 'Make']] = {df_temp.loc[j, 'Model']: {df_temp.loc[j, 'Configuration']: [int(df_temp.loc[j, 'Year'])]}}
            elif df_temp.loc[j, 'Model'] not in dict_vehicle[df_temp.loc[j, 'Make']].keys():
                dict_vehicle[df_temp.loc[j, 'Make']][df_temp.loc[j, 'Model']] = {df_temp.loc[j, 'Configuration']: [int(df_temp.loc[j, 'Year'])]}
            elif df_temp.loc[j, 'Configuration'] not in dict_vehicle[df_temp.loc[j, 'Make']][df_temp.loc[j, 'Model']].keys():
                dict_vehicle[df_temp.loc[j, 'Make']][df_temp.loc[j, 'Model']][df_temp.loc[j, 'Configuration']] = [int(df_temp.loc[j, 'Year'])]
            elif int(df_temp.loc[j, 'Year']) not in dict_vehicle[df_temp.loc[j, 'Make']][df_temp.loc[j, 'Model']][df_temp.loc[j, 'Configuration']]:
                dict_vehicle[df_temp.loc[j, 'Make']][df_temp.loc[j, 'Model']][df_temp.loc[j, 'Configuration']].append(int(df_temp.loc[j, 'Year']))

        # = = = = = = = = = = = = = = =

        list_vehicle = []
        for make in dict_vehicle.keys():
            for model in dict_vehicle[make].keys():
                list_year = []
                for engine in dict_vehicle[make][model].keys():
                    list_year = list(set(list_year + dict_vehicle[make][model][engine]))
                year_begin = min(list_year)
                year_end = max(list_year)
                list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

        input_2.loc[i, 'Vehicle W/o Engine'] = '\n'.join(list_vehicle)

        # = = = = = = = = = = = = = = =

        list_vehicle = []
        for make in dict_vehicle.keys():
            for model in dict_vehicle[make].keys():
                for engine in dict_vehicle[make][model].keys():
                    year_begin = min(dict_vehicle[make][model][engine])
                    year_end = max(dict_vehicle[make][model][engine])
                    list_vehicle.append(f'{make} {model} {year_begin} {engine}'.strip() if year_begin == year_end else f'{make} {model} {year_begin}-{year_end} {engine}'.strip())

        input_2.loc[i, 'Vehicle W/ Engine'] = '\n'.join(list_vehicle)

    # = = = = = = = = = = = = = = =

    output = pd.concat([output, input_2], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

output.to_excel(f'./{param_date}/13.vehicle.xlsx', index=False)

print('Done ~')

part_1-fragment_1.xlsx


Progress: 100%|█████████████████████████| 2470/2470 [00:23<00:00, 107.06it/s]


part_10-fragment_10.xlsx


Progress: 100%|█████████████████████████| 2908/2908 [00:26<00:00, 110.75it/s]


part_11-fragment_11.xlsx


...

part_8-fragment_8.xlsx


Progress: 100%|█████████████████████████| 2940/2940 [00:18<00:00, 160.84it/s]


part_9-fragment_9.xlsx


Progress: 100%|█████████████████████████| 2124/2124 [00:18<00:00, 117.79it/s]


Done ~
